In [1]:
import numpy as np
import random
import pandas as pd

In [2]:
NUM_PLAYERS = 3
random.seed(7)  # for reproducible output

# Simulating one game

In [3]:
# Helper functions
def randSelectPlayer(candidates, exclude=None):
    """
    Randomly selects and returns one entry from the list of candidates.
    If `exclude` is specified, that entry is excluded from the list of candidates.
    """
    choices = list(candidates)
    if exclude:
        choices.remove(exclude)
    
    return random.choice(choices)

def give(giver, receiver, playerWealth):
    """
    Simulates the give action:
    The minimum of the giver's and receiver's wealth is transferred from the giver to the receiver.
    Returns the updated `playerWealth` dictionary, which consists of player:wealth key-value pairs.
    """
    v = playerWealth[giver]
    w = playerWealth[receiver]
    amt = min(v, w)
    playerWealth[receiver] += amt
    playerWealth[giver] -= amt
    return playerWealth, amt
    
def getPlayerByWealth(playerWealth, amt):
    """
    Looks up the `playerWealth` dictionary of player:wealth key-value pairs.
    Returns the player whose value matches `amt` (an integer).
    """
    for p, w in playerWealth.items():
        if w == amt:
            return p
        
    return None
    

def checkGameStatus(playerWealth, winAmt, loseAmt=0):
    """
    Checks if there are any winners and losers.
    - playerWealth: a dictionary of player:wealth key-value pairs.
    - winAmt: an integer representing the winning condition.
    - loseAmt: an integer representing the losing condition (default 0).
    Returns a pair: (winner, loser). The entries are None if they do not yet exist.
    """
    winner = getPlayerByWealth(playerWealth, winAmt)
    loser = getPlayerByWealth(playerWealth, loseAmt)        
    
    return winner, loser

def reportGameResult(winner, losers, numRounds):
    """
    Outputs the game result.
    - winner: the player that is the winner.
    - losers: a list of losers in order, from the first to the last loser.
    Note: By the naming convention, the first entry is the actual loser of the entire game.
    """
    # prints losers in order
    for i in range(len(losers)):
        print(f"Loser {i+1}: {losers[i]}")
    print(f"Winner: {winner}")
    print(f"Number of rounds played: {numRounds}")

def reportGameProgress(numRounds, giver, receiver, transferAmt, playerWealth):
    """
    Outputs the game progress.
    - numRounds: an integer representing the number of rounds played so far.
    - giver: the player that is chosen to give.
    - receiver: the player that is chosen to receive.
    - transferAmt: an integer representing the amount of money transferred from the giver to the receiver.
    - playerWealth: a dictionary of player:wealth key-value pairs, representing each player's current wealth.
    """
    print(f"Round {numRounds}:")
    print(f"Player {giver} gave Player {receiver} ${transferAmt}")
    print(f"Remaining players' wealth: {playerWealth}\n")

In [4]:
# Main function
def playGame(x, y, z, printProgress=False, printResult=False):
    """
    Plays one game.
    - x, y, z: integers representing the initial wealth of Player 1, 2, 3, respectively.
    - printProgress: if True, prints the game progress.
    - printResult: if True, prints the game's result.
    Returns the winner, (first) loser, and number of rounds until the game ends.
    """
    # initialise
    playerWealth = {1: x, 2: y, 3: z}
    numPlayers = NUM_PLAYERS
    total = x + y + z
    winner = None
    losers = []  # losers in order - might be interested in, e.g. P(Player 2 loses before 1)
    numRounds = 0
    
    while numPlayers > 1:
        giver = randSelectPlayer(playerWealth)
        receiver = randSelectPlayer(playerWealth, exclude=giver)
        playerWealth, transferAmt = give(giver, receiver, playerWealth)
        
        winner, loser = checkGameStatus(playerWealth, total)
        # check if a loser has appeared
        if loser:
            losers.append(loser)
            numPlayers -= 1
            playerWealth.pop(loser)
        
        numRounds += 1
        
        if printProgress:
            reportGameProgress(numRounds, giver, receiver, transferAmt, playerWealth)
    
    # sanity checks - at this point, should have 1 winner left, and 2 losers
#     assert(winner)
#     assert(numPlayers == 1)
#     assert(len(losers) == NUM_PLAYERS-1)
    
    if printResult:
        reportGameResult(winner, losers, numRounds)
        
    return winner, losers[0], numRounds

## Examples

In [5]:
playGame(1,2,3, printProgress=True, printResult=True)

Round 1:
Player 2 gave Player 1 $1
Remaining players' wealth: {1: 2, 2: 1, 3: 3}

Round 2:
Player 2 gave Player 1 $1
Remaining players' wealth: {1: 3, 3: 3}

Round 3:
Player 1 gave Player 3 $3
Remaining players' wealth: {3: 6}

Loser 1: 2
Loser 2: 1
Winner: 3
Number of rounds played: 3


(3, 2, 3)

In [6]:
playGame(12,24,36, printProgress=True, printResult=True)

Round 1:
Player 2 gave Player 1 $12
Remaining players' wealth: {1: 24, 2: 12, 3: 36}

Round 2:
Player 3 gave Player 1 $24
Remaining players' wealth: {1: 48, 2: 12, 3: 12}

Round 3:
Player 1 gave Player 2 $12
Remaining players' wealth: {1: 36, 2: 24, 3: 12}

Round 4:
Player 2 gave Player 3 $12
Remaining players' wealth: {1: 36, 2: 12, 3: 24}

Round 5:
Player 1 gave Player 2 $12
Remaining players' wealth: {1: 24, 2: 24, 3: 24}

Round 6:
Player 1 gave Player 3 $24
Remaining players' wealth: {2: 24, 3: 48}

Round 7:
Player 2 gave Player 3 $24
Remaining players' wealth: {3: 72}

Loser 1: 1
Loser 2: 2
Winner: 3
Number of rounds played: 7


(3, 1, 7)

# Simulating the game multiple times and estimating probabilities

In [7]:
# Helper function
def tallyToDf(winners, losers):
    """
    Counts how many times each player has been the winner and loser, from the given tallies
    (lists of entries 1,2,3, representing the player that has won/lost that game).
    Returns the aggregated result as a DataFrame with rows = (winner, loser), columns = Players (1,2,3)
    """
    winnerCount = [(winners.count(i+1)) for i in range(NUM_PLAYERS)]
    loserCount = [(losers.count(i+1)) for i in range(NUM_PLAYERS)]
    data = {"winner": winnerCount, "loser": loserCount}
    return pd.DataFrame.from_dict(data, orient='index', columns=range(1, NUM_PLAYERS+1))

In [8]:
# Main function
def simGame(n, initState, report=True):
    """
    Plays the betting game `n` times, with the provided initial amounts.
    Prints a summary of the game statistics, include:
    - number of games where each player is the winner and loser;
    - estimated probability of player 1 being the winner and loser;
    - average number of rounds played to end a game.
    """
    x, y, z = initState
    winners = []
    firstLosers = []
    numRoundsPlayed = []
    
    for i in range(n):
        winner, firstLoser, numRounds = playGame(x, y, z)
        winners.append(winner)
        firstLosers.append(firstLoser)
        numRoundsPlayed.append(numRounds)
    
    summaryDf = tallyToDf(winners, firstLosers)
    player1Won = summaryDf.loc['winner', 1]
    player1Lost = summaryDf.loc['loser', 1]
    
    if report:
        print(summaryDf)
        print('\n')

        print(f"P(winner = Player 1): {player1Won} / {n} = {player1Won / n}")
        print(f"P(loser = Player 1): {player1Lost} / {n} = {player1Lost / n}")
        print(f"Average number of rounds to end game: {np.mean(numRoundsPlayed)}")
    
    return player1Lost/n

## Examples

In [9]:
simGame(100000, (1,2,3))

            1      2      3
winner  16496  33349  50155
loser   51940  31770  16290


P(winner = Player 1): 16496 / 100000 = 0.16496
P(loser = Player 1): 51940 / 100000 = 0.5194
Average number of rounds to end game: 3.49989


0.5194

In [10]:
simGame(100000, (12,24,36))

            1      2      3
winner  16581  33116  50303
loser   52234  31744  16022


P(winner = Player 1): 16581 / 100000 = 0.16581
P(loser = Player 1): 52234 / 100000 = 0.52234
Average number of rounds to end game: 3.49319


0.52234

In [11]:
simGame(100000, (10,1,1))

            1      2      3
winner  83006   8556   8438
loser    1056  49403  49541


P(winner = Player 1): 83006 / 100000 = 0.83006
P(loser = Player 1): 1056 / 100000 = 0.01056
Average number of rounds to end game: 3.60713


0.01056

In [26]:
MIN = 20
MAX = 30

In [27]:
probs = {}
for x in range(MIN,MAX+1):
    for y in range(MIN,MAX+1):
        for z in range(MIN,MAX+1):
            probs[(x,y,z)] = simGame(10000, (x,y,z), False)

In [28]:
betterGiving1 = []

for x in range(MIN+1,MAX+1):
    for y in range(MIN,MAX+1):
        for z in range(MIN,MAX+1):
            if probs[(x-1,y,z)] < probs[(x,y,z)]:
                betterGiving1.append((x,y,z))
betterGiving1

[(21, 21, 22),
 (21, 22, 22),
 (21, 22, 24),
 (21, 22, 29),
 (21, 22, 30),
 (21, 23, 24),
 (21, 24, 28),
 (21, 25, 22),
 (21, 25, 25),
 (21, 25, 28),
 (21, 26, 23),
 (21, 26, 25),
 (21, 27, 22),
 (21, 27, 24),
 (21, 27, 29),
 (21, 27, 30),
 (21, 29, 22),
 (21, 29, 23),
 (21, 29, 25),
 (21, 29, 26),
 (21, 30, 25),
 (21, 30, 27),
 (21, 30, 28),
 (22, 20, 23),
 (22, 20, 24),
 (22, 20, 26),
 (22, 22, 23),
 (22, 22, 26),
 (22, 23, 23),
 (22, 23, 24),
 (22, 23, 27),
 (22, 23, 28),
 (22, 23, 30),
 (22, 24, 20),
 (22, 24, 23),
 (22, 24, 26),
 (22, 24, 29),
 (22, 24, 30),
 (22, 25, 23),
 (22, 25, 30),
 (22, 26, 26),
 (22, 26, 27),
 (22, 26, 29),
 (22, 26, 30),
 (22, 27, 20),
 (22, 28, 25),
 (22, 28, 28),
 (22, 28, 30),
 (22, 29, 24),
 (22, 29, 28),
 (22, 29, 29),
 (23, 20, 21),
 (23, 20, 28),
 (23, 21, 29),
 (23, 21, 30),
 (23, 24, 24),
 (23, 24, 27),
 (23, 24, 28),
 (23, 25, 24),
 (23, 26, 28),
 (23, 27, 21),
 (23, 27, 25),
 (23, 27, 27),
 (23, 27, 30),
 (23, 28, 27),
 (23, 28, 29),
 (23, 29, 

In [31]:
MIN = 20
MAX = 30

In [32]:
betterGiving1ToOthers = []

for x in range(MIN+1,MAX+1):
    for y in range(MIN,MAX+1):
        for z in range(MIN,MAX+1):
            if (x-1,y+1,z) in probs and (x,y,z) in probs:
                if probs[(x-1,y+1,z)] < probs[(x,y,z)]:
                    betterGiving1ToOthers.append((x,y,z))
            elif (x-1,y,z+1) in probs and (x,y,z) in probs:
                if probs[(x-1,y,z+1)] < probs[(x,y,z)]:
                    betterGiving1ToOthers.append((x,y,z))
betterGiving1ToOthers

[(21, 21, 22),
 (21, 22, 23),
 (21, 22, 24),
 (21, 22, 30),
 (21, 23, 24),
 (21, 24, 25),
 (21, 24, 28),
 (21, 25, 26),
 (21, 25, 29),
 (21, 26, 22),
 (21, 26, 25),
 (21, 26, 27),
 (21, 27, 22),
 (21, 27, 24),
 (21, 27, 28),
 (21, 28, 29),
 (21, 29, 25),
 (21, 29, 30),
 (21, 30, 27),
 (21, 30, 28),
 (21, 30, 29),
 (22, 22, 23),
 (22, 22, 30),
 (22, 23, 24),
 (22, 23, 27),
 (22, 23, 29),
 (22, 24, 20),
 (22, 24, 23),
 (22, 24, 25),
 (22, 24, 26),
 (22, 25, 26),
 (22, 25, 27),
 (22, 25, 29),
 (22, 25, 30),
 (22, 26, 25),
 (22, 26, 27),
 (22, 26, 29),
 (22, 27, 20),
 (22, 27, 28),
 (22, 28, 29),
 (22, 28, 30),
 (22, 29, 30),
 (22, 30, 28),
 (22, 30, 29),
 (23, 20, 21),
 (23, 20, 30),
 (23, 23, 24),
 (23, 24, 25),
 (23, 24, 28),
 (23, 25, 26),
 (23, 25, 28),
 (23, 25, 30),
 (23, 26, 27),
 (23, 26, 28),
 (23, 27, 21),
 (23, 27, 28),
 (23, 28, 20),
 (23, 28, 26),
 (23, 28, 27),
 (23, 28, 29),
 (23, 29, 25),
 (23, 29, 26),
 (23, 29, 27),
 (23, 29, 30),
 (23, 30, 20),
 (23, 30, 24),
 (23, 30, 